In [16]:
import torch 
import torch.nn as nn

In [13]:
! pip3 install torch_optimizer

     |████████████████████████████████| 61 kB 298 kB/s 


In [31]:
import csv
import numpy as np
wine_path = "/content/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",
skiprows=1)
wineq_numpy


array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [81]:
wineq = torch.from_numpy(wineq_numpy)


In [100]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [101]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [102]:
target = wineq[:, -1]
target

tensor([6., 6., 6.,  ..., 6., 7., 6.])

In [103]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6.],
        [6.],
        [6.],
        ...,
        [6.],
        [7.],
        [6.]])

T_c = target
T_u = data 

In [104]:
#splitting the dataset
n_samples = data.shape[0]
n_val = int(0.2 * n_samples)

In [105]:
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]


In [118]:
#train_indices, val_indices

In [119]:
#build training and test set 
train_t_u = data[train_indices]
train_t_c = target_unsqueezed[train_indices]
val_t_u = data[val_indices]
val_t_c = target_unsqueezed[val_indices]

#normalize 
train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u


In [120]:
print(train_t_u.shape)
print(train_t_c.shape)
print(val_t_u.shape)
print(val_t_c.shape)

torch.Size([3919, 11])
torch.Size([3919, 1])
torch.Size([979, 11])
torch.Size([979, 1])


In [121]:
#print(val_t_c)

In [122]:
import torch_optimizer as optim

In [123]:
seq_model = nn.Sequential(
    nn.Linear(11, 25),
    nn.Tanh(),
    nn.Linear(25, 10))
seq_model

Sequential(
  (0): Linear(in_features=11, out_features=25, bias=True)
  (1): Tanh()
  (2): Linear(in_features=25, out_features=10, bias=True)
)

In [124]:
#!pip install torch-optimizer

In [125]:
import torch_optimizer as optim

In [126]:
optimizer = optim.DiffGrad(seq_model.parameters(), lr=0.001)

In [127]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
              t_c_train, t_c_val):
  for epoch in range(1, n_epochs + 1):
      t_p_train = model(t_u_train)
      loss_train = loss_fn(t_p_train, t_c_train)
      t_p_val = model(t_u_val)


      loss_val = loss_fn(t_p_val, t_c_val)
      optimizer.zero_grad()
      loss_train.backward()
      optimizer.step()

      if epoch == 1 or epoch % 1000 == 0:
              print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                    f" Validation loss {loss_val.item():.4f}")


In [129]:
training_loop(
            n_epochs = 7000,
            optimizer = optimizer,
            model = seq_model,
            loss_fn = nn.MSELoss(),
            t_u_train = train_t_u,
            t_u_val = val_t_u,
            t_c_train = train_t_c,
            t_c_val = val_t_c)
print('output', seq_model(val_t_u))
#print('answer', val_t_c)
#print('hidden', seq_model.hidden_linear.weight.grad)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([3919, 1])) that is different to the input size (torch.Size([3919, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([979, 1])) that is different to the input size (torch.Size([979, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Training loss 0.5140, Validation loss 0.5582
Epoch 1000, Training loss 0.5090, Validation loss 0.5569
Epoch 2000, Training loss 0.5050, Validation loss 0.5553
Epoch 3000, Training loss 0.5010, Validation loss 0.5524
Epoch 4000, Training loss 0.4977, Validation loss 0.5506
Epoch 5000, Training loss 0.4958, Validation loss 0.5503
Epoch 6000, Training loss 0.4933, Validation loss 0.5484
Epoch 7000, Training loss 0.4912, Validation loss 0.5459
output tensor([[5.7309, 5.7331, 5.7320,  ..., 5.7317, 5.7316, 5.7322],
        [5.4720, 5.4718, 5.4720,  ..., 5.4716, 5.4714, 5.4718],
        [6.6261, 6.6247, 6.6254,  ..., 6.6258, 6.6260, 6.6255],
        ...,
        [5.1404, 5.1404, 5.1407,  ..., 5.1401, 5.1401, 5.1400],
        [5.3644, 5.3655, 5.3652,  ..., 5.3652, 5.3654, 5.3650],
        [5.8916, 5.8912, 5.8913,  ..., 5.8914, 5.8914, 5.8914]],
       grad_fn=<AddmmBackward0>)
